In [9]:
#Environment setup
! pip install --upgrade xarray zarr gcsfs cftime nc-time-axis bokeh intake-esm cmip6_preprocessing cartopy

  Using cached https://files.pythonhosted.org/packages/e3/25/cc8ccc40d21638ae8514ce2aef1f1db3036e31c2adea797c7501302726fa/xarray-0.15.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/a3/87/383d77399148ef0772da3472b513ecf143252e7c365c51b0f06714800366/zarr-2.4.0.tar.gz
  Using cached https://files.pythonhosted.org/packages/3e/9f/864a9ff497ed4ba12502c4037db8c66fde0049d9dd0388bd55b67e5c4249/gcsfs-0.6.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/53/35/e2fc52247871c51590d6660e684fdc619a93a29f40e3b64894bd4f8c9041/cftime-1.1.0-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/47/2b/4a0681fd7178caa106f5f480217b9381ba77f8f7c8c1e63e91b0fd2cc427/nc_time_axis-1.2.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e0/a7/875aad223b211951a043bf7b0eddcecb8b2afd5131c08945ff07ac968c7f/bokeh-2.0.0.tar.gz
  Using cached https://files.pythonhosted.org/packages/07/84/76cffc7f36433b68f9ee9311

In [2]:
!wget https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
!chmod +x Miniconda3-4.5.4-Linux-x86_64.sh
!bash ./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local

--2020-03-12 18:58:44--  https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.201.79, 104.18.200.79, 2606:4700::6812:c84f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.201.79|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58468498 (56M) [application/x-sh]
Saving to: ‘Miniconda3-4.5.4-Linux-x86_64.sh’

Miniconda3-4.5.4-Li 100%[===================>]  55.76M   144MB/s    in 0.4s    

2020-03-12 18:58:45 (144 MB/s) - ‘Miniconda3-4.5.4-Linux-x86_64.sh’ saved [58468498/58468498]

PREFIX=/usr/local
installing: python-3.6.5-hc3d631a_2 ...
Python 3.6.5 :: Anaconda, Inc.
installing: ca-certificates-2018.03.07-0 ...
installing: conda-env-2.6.0-h36134e3_1 ...
installing: libgcc-ng-7.2.0-hdf63c60_3 ...
installing: libstdcxx-ng-7.2.0-hdf63c60_3 ...
installing: libffi-3.2.1-hd88cf55_4 ...
installing: ncurses-6.1-hf484d3e_0 ...
installing: openssl-1.0.2o-h20670df_0 ...
installing: tk-

In [0]:
import sys
sys.path.append('/usr/local/lib/python3.8/site-packages')

In [8]:
import numpy as np
import pandas as pd
import xarray as xr
import warnings
import matplotlib.pyplot as plt
import intake
import xesmf as xe
%matplotlib inline
import cartopy.crs as ccrs

ModuleNotFoundError: ignored

In [0]:
cat_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
col = intake.open_esm_datastore(cat_url)
col

In [0]:
from cmip6_preprocessing.preprocessing import combined_preprocessing, replace_x_y_nominal_lat_lon, rename_cmip6

def wrapper(ds):
    ds = ds.copy()
    ds = rename_cmip6(ds)
    ds = replace_x_y_nominal_lat_lon(ds)
    return ds

In [0]:

# lets load a bunch more models this time
# load a few models to illustrate the problem
query = dict(experiment_id=['ssp585', 'historical'], table_id='Omon', 
             variable_id='fgco2', grid_label=['gn'], member_id=['r1i1p1f1', 'r1i1p2f1'])
cat = col.search(**query)

print(cat.df['source_id'].unique())
dset_dict = cat.to_dataset_dict(zarr_kwargs={'consolidated': True}, preprocess=combined_preprocessing)

In [0]:
ssp585_dict = {} # dictionary that will hold spliced DataArrays
for name, ds in dset_dict.items(): # Loop through dictionary
    model_name = name.split(".")[2]
    if ('ssp585' not in name): continue
    ssp585_dict[model_name] = ds

his_dict = {} # dictionary that will hold spliced DataArrays
for name, ds in dset_dict.items(): # Loop through dictionary
    model_name = name.split(".")[2]
    if ('historical' not in name) or (model_name not in ssp585_dict.keys()): continue
    his_dict[model_name] = ds

In [0]:
print(his_dict.keys())
print(ssp585_dict.keys())

In [0]:
# setup a common 1 degree global grid 
ds_out = xe.util.grid_global(1, 1)
ds_out 

In [0]:
kgs_to_molyr=1000./12.*3600*24*365

In [0]:
 rm_his_dict = {}
for name, ds in his_dict.items():
    print(name)
    scale=1 if name != 'BCC-CSM2-MR' else -1 *12/44
    regridder = xe.Regridder(ds, ds_out, 'bilinear', ignore_degenerate=True)
    ds_in = ds.sel(time=slice('1980-01-01','2014-12-31')).mean(dim='time')*kgs_to_molyr*scale
    dsrm = regridder(ds_in).compute()
    rm_his_dict[name] = dsrm

In [0]:
rm_ssp585_dict = {}
for name, ds in ssp585_dict.items():
    print(name)
    scale=1 if name != 'BCC-CSM2-MR' else -1 *12/44.
    regridder = xe.Regridder(ds, ds_out, 'bilinear', ignore_degenerate=True)
    ds_in = ds.sel(time=slice('2045-01-01','2054-12-31')).mean(dim='time')*kgs_to_molyr*scale
    dsrm = regridder(ds_in).compute()
    rm_ssp585_dict[name] = dsrm

In [0]:
vmin=-4
vmax=4
fig, axa = plt.subplots(3,3, figsize=(12,12))
for i, (name, ds) in enumerate(rm_his_dict.items()):
    print(name)
    scale=1
    ax=axa.flat[i]
    (ds['fgco2'].isel(member_id=0)).plot(ax=ax, cmap='RdBu_r', vmin=vmin, vmax=vmax)
    ax.set_title(name)
    

In [0]:
fig, axa = plt.subplots(3,3, figsize=(12,12))
for i, (name, ds) in enumerate(rm_ssp585_dict.items()):
    print(name)
    scale=1
    ax=axa.flat[i]
    (ds['fgco2'].isel(member_id=0)).plot(ax=ax, cmap='RdBu_r', vmin=vmin, vmax=vmax)
    ax.set_title(name)

In [0]:
meanf = rm_ssp585_dict['BCC-CSM2-MR']['fgco2'].values*0.0

fig, axa = plt.subplots(3,3, figsize=(12,12))
for i, (name, ds) in enumerate(rm_ssp585_dict.items()):
    print(name)
    ax=axa.flat[i]
    (ds['fgco2'].isel(member_id=0) - rm_his_dict[name]['fgco2'].isel(member_id=0)).plot(ax=ax, cmap='RdBu_r', vmin=vmin, vmax=vmax)
    ax.set_title(name)
    if name != 'BCC-CSM2-MR':
        meanf = meanf + (ds['fgco2'].isel(member_id=0) - rm_his_dict[name]['fgco2'].isel(member_id=0)).compute().values

meanf=meanf/(i-1)
ax = axa.flat[i+1]
ax.pcolormesh(meanf.squeeze(), cmap='RdBu_r', vmin=vmin, vmax=vmax)    

In [0]:
fig = plt.figure(figsize=(12,12))
plt.pcolormesh(meanf.squeeze(), cmap='RdBu_r', vmin=vmin, vmax=vmax)  
plt.xlim([0,100])
plt.ylim([100,160])

In [0]:
fig = plt.figure(figsize=(18,12))

vmin=-5
vmax=5
ncol=15

axo = plt.subplot(1, 2, 1, projection=ccrs.Robinson(central_longitude=260))

cbl = axo.pcolormesh(fgco2_obs.lon.values, fgco2_obs.lat.values, fgco2_obs.mean(dim='time').values*1000./12*sec2yr, cmap=plt.cm.get_cmap('RdBu_r', ncol), 
                    vmin=vmin, vmax=vmax, rasterized=True,transform=ccrs.PlateCarree())

divider = make_axes_locatable(axo)
ax_cb = divider.append_axes('bottom', size="5%", pad=0.1, axes_class=plt.Axes)

_=plt.colorbar(cbl, cax=ax_cb, label=r'Ocean carbon flux (mol m$^{-2}$ yr$^{-1}$)', orientation='horizontal')
axo.set_title('Carbon flux climatology:\n Obs. (Landschutzer 2015))')
axo.coastlines()


axr = plt.subplot(1, 2, 2, projection=ccrs.Robinson(central_longitude=260))

cbr = axr.pcolormesh(fgco2_obs.lon.values, fgco2_obs.lat.values, dsc.mean(axis=0)*1000./12*sec2yr, 
                     cmap=plt.cm.get_cmap('RdBu_r', ncol), vmin=vmin, vmax=vmax, rasterized=True,transform=ccrs.PlateCarree())


axr.set_title('Carbon flux climatology:\n CMIP5 / historical (1980-2005)')
axr.coastlines()

plt.savefig('global_ocean_carbon_flux_cmip5-obs.png', bbox_inches='tight', dpi=300)